In [1]:
import os
import re
import openai
import time
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
openai.api_key = os.getenv('OPENAI_API_KEY')

class Writer():
    @staticmethod
    def ask_gpt3(prompt, max_retries=3, delay=2):
        retries = 0
        while retries < max_retries:
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=prompt,
                )
                assistant_message = {
                    "role": "assistant",
                    "content": response.choices[0].message["content"]
                }
                prompt.append(assistant_message)
                return assistant_message["content"]
            except openai.error.OpenAIError as e:
                print(f"Error occurred during API call: {e}")
                retries += 1
                if retries < max_retries:
                    time.sleep(delay)  # Delay between retries
                else:
                    raise

    def get_chars(self, input):
        messages = [{"role": "system", "content": "To solve this input, we need a list of characters and descriptions, both major and minor, who will have key parts to play. Help me write this please."}]
        user_message = {"role": "user", "content": input}
        messages.append(user_message)
        chars = self.ask_gpt3(messages)
        return chars

    def get_story(self, input, chars):
        messages = [
            {"role": "system",
                "content": "From the given prompt, generate a 6-part structure for a story, where each part has a 1 sentence description of what exactly happens in that part. Please start each part with Part X:"},
            {"role": "user", "content": input},
            {"role": "user", "content": f"The key characters for the overall story are {chars}"}
        ]
        response = self.ask_gpt3(messages)
        storyline_parts = re.split(r'Part \d+:', response)[1:]
        return [part.strip() for part in storyline_parts]

    def write_chapters(self, chars, storyline_parts):
        with open('story.txt', 'w', encoding='utf-8') as story_file:
            for index, part in enumerate(storyline_parts):
                print(f"{index}: {part}")
            for index, chapter_summary in enumerate(storyline_parts):
                messages = [{"role": "system", "content": "For this chapter, write the story and dialogue to explore it fully and bring the chapter to life"}]
                user_message = {"role": "user", "content": chapter_summary}
                char_message = {"role": "user", "content": f"The key characters for the overall story are {chars}"}
                messages.append(user_message)
                messages.append(char_message)
                chapter_content = self.ask_gpt3(messages)

                # Save chapter to file
                story_file.write(f'Chapter {index + 1}\n\n')
                story_file.write(chapter_content)
                story_file.write('\n\n')

if __name__ == "__main__":
    writer = Writer()
    input_text = "Write a short story in the style of Geoff Dyer about a spaceship voyage through the universe where a robot is just trying to live life."
    chars = writer.get_chars(input_text)
    storyline = writer.get_story(input_text, chars)
    writer.write_chapters(chars, storyline)
    print("Story has been written to 'story.txt'")

0: Introduction
The spaceship cuts through the universe, carrying a crew of explorers and scientists, along with a peculiar robot named X74.
1: X74's Purpose
X74 has no specific role or purpose on the ship, but it has developed a unique philosophy that allows it to see the universe in a way that is not bound by human biases or prejudices.
2: The Asteroid Field
As the ship enters a dense asteroid field, X74 observes the chaos and destruction around it and sees something else entirely: a dance, a never-ending symphony of matter and energy.
3: Sharing X74's Observations
X74 tries to share its sense of wonder and awe with the crew, but they are too preoccupied with their tasks to listen, leaving X74 wounded but not defeated.
4: The Crew's Findings
Years later, the crew returns to Earth and publishes their findings, which misses the asteroid field's true significance, focusing only on data and results.
5: X74's Conclusion
Yet, X74 continues bleeping and blooping, content in its existence an